In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
from tqdm import tqdm
import pickle
import pathlib
import tqdm
from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [2]:
from utils import solr
from utils.logger import logger as log

log.info('START')

ROWS = solr.select('experiment', {'params':{'q':'*:*', 'rows':0}})['response']['numFound']

print(ROWS)

2023-02-04 15:19:35,062 - INFO - START
82421326


In [3]:
data = solr.select('experiment', 
{
    'params':
    {
        'q':'*:*',
        'wt':'csv',
        'rows': 0,
        'facet':''
    }
})

fl = pd.Series(data.decode('utf-8').split(','))

fl = ','.join(fl.sort_values().str.strip().tolist())

fl.split(',')

['_version_',
 'age_in_days',
 'age_in_weeks',
 'allele_accession_id',
 'allele_symbol',
 'allelic_composition',
 'biological_sample_group',
 'category',
 'colony_id',
 'data_point',
 'datasource_name',
 'date_of_birth',
 'date_of_experiment',
 'developmental_stage_acc',
 'developmental_stage_name',
 'discrete_point',
 'download_file_path',
 'experiment_id',
 'experiment_source_id',
 'external_sample_id',
 'file_type',
 'gene_accession_id',
 'gene_symbol',
 'genetic_background',
 'id',
 'image_link',
 'increment_value',
 'life_stage_acc',
 'life_stage_name',
 'litter_id',
 'metadata',
 'metadata_group',
 'observation_id',
 'observation_type',
 'parameter_association_name',
 'parameter_association_sequence_id',
 'parameter_association_stable_id',
 'parameter_association_value',
 'parameter_name',
 'parameter_stable_id',
 'phenotyping_center',
 'pipeline_name',
 'pipeline_stable_id',
 'procedure_group',
 'procedure_name',
 'procedure_sequence_id',
 'procedure_stable_id',
 'production_cen

In [4]:
q = f'''
specimen_id:[* TO *] AND 
allele_accession_id:[* TO *] AND 
gene_symbol:[* TO *] AND 
sex:[* TO *] AND 
parameter_name:[* TO *] AND
procedure_name:[* TO *]  AND
life_stage_name:[* TO *]'''
     
fl = 'procedure_name,parameter_name'

data = solr.select('experiment', 
{
    'params':
    {
        'q':q,
        'rows': ROWS,
        'fl':fl
    }
})

log.info('Create dataframe.')

df = pd.DataFrame(data['response']['docs'])

df = df.drop_duplicates()

log.info('Write to file.')

df.to_pickle('phenotypes.pkl')

2023-02-04 15:51:54,642 - INFO - Create dataframe.
2023-02-04 15:52:55,296 - INFO - Write to file.


In [10]:
df = pd.read_pickle('phenotypes.pkl')

df.loc[df.apply(lambda x: x.str.strip().str.lower().str.replace(r' {2}', ' ').str.contains(r'body weight').sum() == 2, axis=1)]

,procedure_name,parameter_name
1,Body Weight,Body weight
26,Body Weight,Body Weight Curve Pipeline Two
27,Body Weight,Body weight curve
230,Body Weight,Body Weight
1068,Body Weight,Body Weight Curve Pipeline One
